# Exploratory Notebook

In [42]:
import pandas as pd
import sqlite3

In [43]:
budgets_df = pd.read_csv("../Data/tn.movie_budgets.csv.gz")

In [44]:
budgets_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [45]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [46]:
budgets_df.describe()

,id
count,5782.000000
mean,50.372363
std,28.821076
min,1.000000
25%,25.000000
50%,50.000000
75%,75.000000
max,100.000000


In [47]:
tmdb_df = pd.read_csv("../Data/tmdb.movies.csv.gz")

In [48]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [49]:
tmdb_df['popularity'].max()

80.773

In [50]:
tmdb_df = tmdb_df.sort_values('popularity', ascending=False)[0:500]

In [51]:
tmdb_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
23811,23811,"[12, 28, 14]",299536,en,Avengers: Infinity War,80.773,2018-04-27,Avengers: Infinity War,8.3,13948
11019,11019,"[28, 53]",245891,en,John Wick,78.123,2014-10-24,John Wick,7.2,10081
23812,23812,"[28, 12, 16, 878, 35]",324857,en,Spider-Man: Into the Spider-Verse,60.534,2018-12-14,Spider-Man: Into the Spider-Verse,8.4,4048
11020,11020,"[28, 12, 14]",122917,en,The Hobbit: The Battle of the Five Armies,53.783,2014-12-17,The Hobbit: The Battle of the Five Armies,7.3,8392
5179,5179,"[878, 28, 12]",24428,en,The Avengers,50.289,2012-05-04,The Avengers,7.6,19673
...,...,...,...,...,...,...,...,...,...,...
14226,14226,"[18, 53, 36]",314365,en,Spotlight,16.354,2015-11-06,Spotlight,7.8,4935
23948,23948,"[10749, 35]",503616,en,Second Act,16.322,2018-12-21,Second Act,6.2,361
7913,7913,[35],4258,en,Scary Movie 5,16.316,2013-04-12,Scary Movie 5,4.7,1494
23949,23949,"[27, 53]",467660,en,Unsane,16.316,2018-03-23,Unsane,6.2,667


In [52]:
tmdb_df = tmdb_df.drop(['genre_ids', 'id'], axis=1)

In [53]:
tmdb_df

,Unnamed: 0,original_language,original_title,popularity,release_date,title,vote_average,vote_count
23811,23811,en,Avengers: Infinity War,80.773,2018-04-27,Avengers: Infinity War,8.3,13948
11019,11019,en,John Wick,78.123,2014-10-24,John Wick,7.2,10081
23812,23812,en,Spider-Man: Into the Spider-Verse,60.534,2018-12-14,Spider-Man: Into the Spider-Verse,8.4,4048
11020,11020,en,The Hobbit: The Battle of the Five Armies,53.783,2014-12-17,The Hobbit: The Battle of the Five Armies,7.3,8392
5179,5179,en,The Avengers,50.289,2012-05-04,The Avengers,7.6,19673
...,...,...,...,...,...,...,...,...
14226,14226,en,Spotlight,16.354,2015-11-06,Spotlight,7.8,4935
23948,23948,en,Second Act,16.322,2018-12-21,Second Act,6.2,361
7913,7913,en,Scary Movie 5,16.316,2013-04-12,Scary Movie 5,4.7,1494
23949,23949,en,Unsane,16.316,2018-03-23,Unsane,6.2,667


In [54]:
tmdb_df['original_language'].value_counts()

en    479
ja      8
es      4
de      3
fr      3
zh      1
ar      1
nl      1
Name: original_language, dtype: int64

In [55]:
tmdb_df = tmdb_df.loc[tmdb_df['original_language'] == "en"]

In [56]:
tmdb_df['original_title'].value_counts()

Room                              2
Call Me by Your Name              2
Goosebumps                        2
The Revenant                      2
Sing                              2
                                 ..
Southpaw                          1
Under the Silver Lake             1
Jurassic World: Fallen Kingdom    1
Overlord                          1
Thor                              1
Name: original_title, Length: 430, dtype: int64

In [57]:
tmdb_df = tmdb_df.drop_duplicates(subset=['original_title'])

In [59]:
tmdb_df

,Unnamed: 0,original_language,original_title,popularity,release_date,title,vote_average,vote_count
23811,23811,en,Avengers: Infinity War,80.773,2018-04-27,Avengers: Infinity War,8.3,13948
11019,11019,en,John Wick,78.123,2014-10-24,John Wick,7.2,10081
23812,23812,en,Spider-Man: Into the Spider-Verse,60.534,2018-12-14,Spider-Man: Into the Spider-Verse,8.4,4048
11020,11020,en,The Hobbit: The Battle of the Five Armies,53.783,2014-12-17,The Hobbit: The Battle of the Five Armies,7.3,8392
5179,5179,en,The Avengers,50.289,2012-05-04,The Avengers,7.6,19673
...,...,...,...,...,...,...,...,...
17440,17440,en,Spotlight,16.354,2015-11-06,Spotlight,7.8,4935
23948,23948,en,Second Act,16.322,2018-12-21,Second Act,6.2,361
7913,7913,en,Scary Movie 5,16.316,2013-04-12,Scary Movie 5,4.7,1494
23949,23949,en,Unsane,16.316,2018-03-23,Unsane,6.2,667


In [8]:
conn = sqlite3.connect('../Data/im.db')
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print(tables)

            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers
